In [56]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import api_keys
from spotipy.oauth2 import SpotifyOAuth

In [57]:
# https://spotipy.readthedocs.io/en/2.19.0/#redirect-uri

In [58]:
#Authentication 
client_credential_manager = SpotifyClientCredentials(client_id = api_keys.client_id,
                                                                                client_secret=api_keys.client_secret)

# sp = spotipy.Spotify(client_credentials_manager=client_credential_manager)

In [77]:
scope = "user-library-read,playlist-read-private"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id="adea15fa489d4c38b417367ddbdb85b2", client_secret="b3a5c5daf15b4493977c17ddd0b05bf5", redirect_uri="http://localhost"))

##loop through intervals of 50 to hit the api endpoint with to retrievbe songs in buckets of 50
## generate a list of all increments of 50 up to 5000
results_temp = sp.current_user_saved_tracks(limit=50, offset=0)

In [122]:
for i in results_temp:
    print(i)

href
items
limit
next
offset
previous
total


In [136]:
### Items to add:
# added_at_to_my_playlist: results_temp["items"][0]["added_at"]
# artist_name: results_temp["items"][0]["track"]["album"]["artists"][0]["name"]
# unique_artist_id : results_temp["items"][0]["track"]["album"]["artists"][0]["uri"]
# release_date: results_temp["items"][0]["track"]["album"]["release_date"]
# song_name : results_temp["items"][0]["track"]["name"]
# song_lengh_seconds : int(results_temp["items"][0]["track"]["duration_ms"])/1000
# song_explicit : results_temp["items"][0]["track"]["explicit"]
# song_linkId : results_temp["items"][0]["track"]["external_urls"]["spotify"]
# song_popularity : results_temp["items"][0]["track"]["popularity"]
# album_name : results_temp["items"][1]["track"]["album"]["name"]
# spotify_uri_track : results_temp["items"][0]["track"]["album"]["uri"]

'2023-06-02T02:50:17Z'

In [93]:
# results_temp["items"]["release_date"]
for i in results_temp["items"][0]:
    print(i)

added_at
track


In [74]:
increment_of50_list = [i for i in range(0, 5000, 50)]
increment_of50_list_test = [i for i in range(0, 300, 50)]
data = {}

while results["next"] is not None:
    for i in increment_of50_list:
        results = sp.current_user_saved_tracks(limit=50, offset=i)
        # added_at_to_my_playlist: results_temp["items"][0]["added_at"]
        # artist_name: results_temp["items"][0]["track"]["album"]["artists"][0]["name"]
        # unique_artist_id : results_temp["items"][0]["track"]["album"]["artists"][0]["uri"]
        # release_date: results_temp["items"][0]["track"]["album"]["release_date"]
        # song_name : results_temp["items"][0]["track"]["name"]
        # song_lengh_seconds : int(results_temp["items"][0]["track"]["duration_ms"])/1000
        # song_explicit : results_temp["items"][0]["track"]["explicit"]
        # song_linkId : results_temp["items"][0]["track"]["external_urls"]["spotify"]
        # song_popularity : results_temp["items"][0]["track"]["popularity"]
        # album_name : results_temp["items"][1]["track"]["album"]["name"]
        # spotify_uri_track : results_temp["items"][0]["track"]["album"]["uri"]


'https://api.spotify.com/v1/me/tracks?offset=50&limit=50'

In [67]:
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

0 Wiley Beckett  –  Disco Nights
1 Ozer  –  Polo Fendi
2 Young Franco  –  Juice
3 Jungle  –  Romeo
4 Kota the Friend  –  Red Lights (feat. Hello O'shay)
5 3LAU  –  Better With You - Kastra & twoDB Remix
6 Boombox Cartel  –  All I Want
7 Man Cub  –  Good Enough
8 Morgan Wallen  –  You Proof
9 Tory Lanez  –  ‘87 Stingray
10 Tory Lanez  –  Hurt From Mercury
11 Richy Mitch & The Coal Miners  –  Sage
12 Masego  –  What You Wanna Try
13 Marshmello  –  Fell In Love
14 Paul Russell  –  Ms. Poli Sci
15 NLE Choppa  –  DO IT AGAIN (feat. 2Rare)
16 Jelly Roll  –  Need A Favor
17 Luke Combs  –  Beautiful Crazy
18 Ghost Riders  –  Happy Now
19 Route 33  –  Simple Things
20 Duncan Toombs  –  A Gift
21 Tyler, The Creator  –  DOGTOOTH
22 Lil Tecca  –  Blessing
23 Trippie Redd  –  1st Degree Murder
24 Mariah the Scientist  –  Only Human
25 Morgan Evans  –  Over For You
26 San Holo  –  DON'T LOOK DOWN (feat. Lizzy Land)
27 San Holo  –  We Rise
28 The Blaze  –  BLOOM
29 The Blaze  –  HAZE
30 boygenius  – 